One should:
- Download Nexus data ()
- Launch entitycore (make run-docker)
- Load Nexus data ()
- Launch obi-one (make run-local)
- Find a morphology with an ID

In [1]:
import requests
from obi_auth import get_token
from entitysdk.client import Client
from entitysdk.common import ProjectContext

base_url = "http://127.0.0.1:8100"

virtual_lab_id="e6030ed8-a589-4be2-80a6-f975406eb1f6"
project_id="2720f785-a3a2-4472-969d-19a53891c817"
token = get_token(environment="staging")
project_context = ProjectContext.from_vlab_url(
            f"https://staging.openbraininstitute.org/app/virtual-lab/lab/{virtual_lab_id}/project/{project_id}/home"
        )
client = Client(environment="staging", project_context=project_context)



In [2]:
from entitysdk.models.morphology import (
    ReconstructionMorphology,
)
kwargs = {}
client.search_entity(
    entity_type=ReconstructionMorphology, query=kwargs, token=token, limit=10
).all()

[ReconstructionMorphology(assets=[Asset(id=UUID('d0597874-a57b-462b-b057-bc94ad1730c7'), update_date=None, creation_date=None, path='18864_00528.swc', full_path='public/a98b7abc-fc46-4700-9e3d-37137812c730/0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6/assets/reconstruction_morphology/e380cafe-c36b-43e1-b040-5c7ee34b0e72/18864_00528.swc', is_directory=False, content_type='application/swc', size=3731439, sha256_digest='0bad607f51a1206760bfe79a43593325935a90d70de6ad593ca68c4648c0c0ee', status='created', meta={'legacy': {'name': '18864_00528.swc', '@type': 'DataDownload', 'digest': {'value': '0bad607f51a1206760bfe79a43593325935a90d70de6ad593ca68c4648c0c0ee', 'algorithm': 'SHA-256'}, 'atLocation': {'@type': 'Location', 'store': {'@id': 'https://bbp.epfl.ch/neurosciencegraph/data/storages/proj166_storage', '_rev': 1, '@type': 'RemoteDiskStorage'}, 'location': 'file:///gpfs/bbp.cscs.ch/data/project/proj166/nexus/bbp-external/seu/d/b/b/6/6/2/b/b/18864_00528.swc'}, 'contentUrl': 'https://bbp.epfl.ch/nex

In [3]:
from uuid import UUID
reconstruction_morphology_id = UUID("305d3a94-dc7c-4c8f-80ca-ced67e9d6099")

In [4]:
# Construct the full endpoint URL
url = f"{base_url}/declared/neuron-morphology-metrics/{reconstruction_morphology_id}"

# Set headers
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/json"
}

# Optionally include optional headers if they are set
if virtual_lab_id:
    headers["virtual-lab-id"] = virtual_lab_id
if project_id:
    headers["project-id"] = project_id

# Make the GET request
response = requests.get(url, headers=headers)

# Check the response
if response.status_code == 200:
    data = response.json()
    print("Success:", data)
else:
    print(f"Error {response.status_code}: {response.text}")


Error 404: {"detail":{"code":"NOT_FOUND","detail":"Reconstruction morphology 305d3a94-dc7c-4c8f-80ca-ced67e9d6099 not found."}}
